# 0. Imports

In [1]:
# Basic imports
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt

# Functional imports
import json
import pyarrow
from pickle import dump

# Model imports
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import f_classif, SelectKBest

# 1. Data Collection

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df_2018 = pd.read_parquet("/workspace/final_project/data/raw/Combined_Flights_2018.parquet", engine="pyarrow")
df_2019 = pd.read_parquet("/workspace/final_project/data/raw/Combined_Flights_2019.parquet", engine="pyarrow")
df_2020 = pd.read_parquet("/workspace/final_project/data/raw/Combined_Flights_2020.parquet", engine="pyarrow")
df_2021 = pd.read_parquet("/workspace/final_project/data/raw/Combined_Flights_2021.parquet", engine="pyarrow")
df_2022 = pd.read_parquet("/workspace/final_project/data/raw/Combined_Flights_2022.parquet", engine="pyarrow")

In [4]:
data_frames = [df_2018, df_2019, df_2020, df_2021, df_2022]
df = pd.concat(data_frames, ignore_index=True)

In [5]:
df.head()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2018-01-23,Endeavor Air Inc.,ABY,ATL,False,False,1202,1157.0,0.0,-5.0,1256.0,0.0,38.0,62.0,59.0,145.0,2018,1,1,23,2,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8928A,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1200-1259,14.0,1211.0,1249.0,7.0,1304,-8.0,0.0,-1.0,1300-1359,1,0.0
1,2018-01-24,Endeavor Air Inc.,ABY,ATL,False,False,1202,1157.0,0.0,-5.0,1258.0,0.0,36.0,62.0,61.0,145.0,2018,1,1,24,3,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N800AY,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1200-1259,13.0,1210.0,1246.0,12.0,1304,-6.0,0.0,-1.0,1300-1359,1,0.0
2,2018-01-25,Endeavor Air Inc.,ABY,ATL,False,False,1202,1153.0,0.0,-9.0,1302.0,0.0,40.0,62.0,69.0,145.0,2018,1,1,25,4,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8836A,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1200-1259,18.0,1211.0,1251.0,11.0,1304,-2.0,0.0,-1.0,1300-1359,1,0.0
3,2018-01-26,Endeavor Air Inc.,ABY,ATL,False,False,1202,1150.0,0.0,-12.0,1253.0,0.0,35.0,62.0,63.0,145.0,2018,1,1,26,5,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N800AY,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1200-1259,17.0,1207.0,1242.0,11.0,1304,-11.0,0.0,-1.0,1300-1359,1,0.0
4,2018-01-27,Endeavor Air Inc.,ABY,ATL,False,False,1400,1355.0,0.0,-5.0,1459.0,0.0,36.0,60.0,64.0,145.0,2018,1,1,27,6,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8839E,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1400-1459,17.0,1412.0,1448.0,11.0,1500,-1.0,0.0,-1.0,1500-1559,1,0.0


# 2.2 Exploration and Data cleaning

# Explicación de las columnas de datos de vuelos

### Información del vuelo
- **FlightDate**: Fecha del vuelo (año, mes y día).
- **Airline**: Código de la aerolínea que opera el vuelo.
- **Origin**: Código del aeropuerto de origen (IATA).
- **Dest**: Código del aeropuerto de destino (IATA).

### Indicadores de interrupción
- **Cancelled**: Indica si el vuelo fue cancelado (1 = Sí, 0 = No).
- **Diverted**: Indica si el vuelo fue desviado a otro aeropuerto (1 = Sí, 0 = No).

### Horarios de salida
- **CRSDepTime**: Hora programada de salida (horario planificado).
- **DepTime**: Hora real de salida del vuelo.
- **DepDelayMinutes**: Minutos de retraso en la salida.
- **DepDelay**: Retraso en la salida (valores negativos indican salida anticipada).

### Horarios de llegada
- **ArrTime**: Hora real de llegada del vuelo.
- **ArrDelayMinutes**: Minutos de retraso en la llegada.
- **ArrDelay**: Retraso en la llegada (valores negativos indican llegada anticipada).

### Duración del vuelo
- **AirTime**: Tiempo efectivo en el aire, en minutos.
- **CRSElapsedTime**: Tiempo total estimado del vuelo (planificado).
- **ActualElapsedTime**: Tiempo total real del vuelo.

### Distancia y ubicación
- **Distance**: Distancia entre los aeropuertos de origen y destino, en millas.

### Fechas y tiempo
- **Year**: Año del vuelo.
- **Quarter**: Trimestre del año (1 a 4).
- **Month**: Mes del año (1 a 12).
- **DayofMonth**: Día del mes.
- **DayOfWeek**: Día de la semana (1 = Lunes, 7 = Domingo).

### Información de la aerolínea
- **Marketing_Airline_Network**: Aerolínea responsable de vender el vuelo.
- **Operated_or_Branded_Code_Share_Partners**: Aerolíneas asociadas que operan el vuelo.
- **DOT_ID_Marketing_Airline**: Identificación del Departamento de Transporte (DOT) de la aerolínea de marketing.
- **IATA_Code_Marketing_Airline**: Código IATA de la aerolínea de marketing.
- **Flight_Number_Marketing_Airline**: Número del vuelo asignado por la aerolínea de marketing.
- **Operating_Airline**: Aerolínea que opera físicamente el vuelo.
- **DOT_ID_Operating_Airline**: Identificación del DOT de la aerolínea operativa.
- **IATA_Code_Operating_Airline**: Código IATA de la aerolínea operativa.

### Información de la aeronave
- **Tail_Number**: Matrícula o identificador único de la aeronave.
- **Flight_Number_Operating_Airline**: Número del vuelo asignado por la aerolínea operativa.

### Información del aeropuerto de origen
- **OriginAirportID**: Identificador único del aeropuerto de origen.
- **OriginAirportSeqID**: Secuencia única del aeropuerto de origen.
- **OriginCityMarketID**: Identificador de mercado de la ciudad de origen.
- **OriginCityName**: Nombre de la ciudad del aeropuerto de origen.
- **OriginState**: Abreviación del estado del aeropuerto de origen.
- **OriginStateFips**: Código FIPS del estado del aeropuerto de origen.
- **OriginStateName**: Nombre completo del estado del aeropuerto de origen.
- **OriginWac**: Código WAC (World Area Code) del aeropuerto de origen.

### Información del aeropuerto de destino
- **DestAirportID**: Identificador único del aeropuerto de destino.
- **DestAirportSeqID**: Secuencia única del aeropuerto de destino.
- **DestCityMarketID**: Identificador de mercado de la ciudad de destino.
- **DestCityName**: Nombre de la ciudad del aeropuerto de destino.
- **DestState**: Abreviación del estado del aeropuerto de destino.
- **DestStateFips**: Código FIPS del estado del aeropuerto de destino.
- **DestStateName**: Nombre completo del estado del aeropuerto de destino.
- **DestWac**: Código WAC (World Area Code) del aeropuerto de destino.

### Indicadores de retraso en la salida
- **DepDel15**: Indica si el vuelo tuvo un retraso mayor a 15 minutos en la salida (1 = Sí, 0 = No).
- **DepartureDelayGroups**: Clasificación del retraso en la salida por grupos de tiempo.
- **DepTimeBlk**: Bloque horario de la salida (por ejemplo, "0700-0759").

### Información de movimientos de la aeronave
- **TaxiOut**: Tiempo en minutos que la aeronave estuvo en la pista antes de despegar.
- **WheelsOff**: Hora en que las ruedas dejaron la pista durante el despegue.
- **WheelsOn**: Hora en que las ruedas tocaron la pista durante el aterrizaje.
- **TaxiIn**: Tiempo en minutos que la aeronave estuvo en la pista después de aterrizar.

### Indicadores de retraso en la llegada
- **CRSArrTime**: Hora programada de llegada (horario planificado).
- **ArrDel15**: Indica si el vuelo tuvo un retraso mayor a 15 minutos en la llegada (1 = Sí, 0 = No).
- **ArrivalDelayGroups**: Clasificación del retraso en la llegada por grupos de tiempo.
- **ArrTimeBlk**: Bloque horario de la llegada (por ejemplo, "0700-0759").

### Otros indicadores
- **DistanceGroup**: Clasificación de la distancia del vuelo por grupos.
- **DivAirportLandings**: Número de aterrizajes no programados en aeropuertos de desvío.

In [6]:
print('Our dataframe contains {} rows and it has {} features.'.format(len(df), df.shape[1]))

Our dataframe contains 29193782 rows and it has 61 features.


---

## 2.2 Identifying Duplicated and Null Values

In [7]:
df.describe()

,FlightDate,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginStateFips,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestStateFips,DestWac,DepDel15,DepartureDelayGroups,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,DistanceGroup,DivAirportLandings
count,29193782,2.919378e+07,2.843213e+07,2.843070e+07,2.843070e+07,2.840760e+07,2.834760e+07,2.834122e+07,2.919376e+07,2.834814e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.919378e+07,2.843070e+07,2.843070e+07,2.841322e+07,2.841323e+07,2.840065e+07,2.840064e+07,2.919378e+07,2.834760e+07,2.834760e+07,2.834760e+07,2.919378e+07,2.919369e+07
mean,2020-04-23 22:27:03.485606,1.326262e+03,1.329296e+03,1.278311e+01,9.306866e+00,1.468046e+03,1.280992e+01,1.090875e+02,1.387605e+02,1.332941e+02,7.797346e+02,2.019829e+03,2.448568e+00,6.327840e+00,1.575161e+01,3.974879e+00,1.982793e+04,2.691383e+03,2.000173e+04,2.691343e+03,1.267630e+04,1.267634e+06,3.176109e+04,2.729479e+01,5.478744e+01,1.267628e+04,1.267632e+06,3.176111e+04,2.729502e+01,5.478760e+01,1.727673e-01,-1.609496e-02,1.671094e+01,1.352845e+03,1.463262e+03,7.532169e+00,1.489004e+03,3.609370e+00,1.766207e-01,-2.887085e-01,3.593297e+00,3.505826e-03
min,2018-01-01 00:00:00,1.000000e+00,1.000000e+00,0.000000e+00,-1.280000e+03,1.000000e+00,0.000000e+00,-1.244000e+03,-2.920000e+02,-1.228000e+03,1.600000e+01,2.018000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.939300e+04,1.000000e+00,1.939300e+04,1.000000e+00,1.013500e+04,1.013505e+06,3.007000e+04,1.000000e+00,1.000000e+00,1.013500e+04,1.013505e+06,3.007000e+04,1.000000e+00,1.000000e+00,0.000000e+00,-2.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,-1.290000e+03,0.000000e+00,-2.000000e+00,1.000000e+00,0.000000e+00
25%,2019-03-18 00:00:00,9.180000e+02,9.200000e+02,0.000000e+00,-6.000000e+00,1.055000e+03,0.000000e+00,5.900000e+01,8.800000e+01,8.200000e+01,3.540000e+02,2.019000e+03,1.000000e+00,3.000000e+00,8.000000e+00,2.000000e+00,1.979000e+04,1.100000e+03,1.979000e+04,1.100000e+03,1.129200e+04,1.129202e+06,3.069300e+04,1.200000e+01,3.400000e+01,1.129200e+04,1.129202e+06,3.069300e+04,1.200000e+01,3.400000e+01,0.000000e+00,-1.000000e+00,1.100000e+01,9.350000e+02,1.051000e+03,4.000000e+00,1.108000e+03,-1.600000e+01,0.000000e+00,-2.000000e+00,2.000000e+00,0.000000e+00
50%,2020-02-08 00:00:00,1.320000e+03,1.323000e+03,0.000000e+00,-3.000000e+00,1.505000e+03,0.000000e+00,9.100000e+01,1.210000e+02,1.160000e+02,6.260000e+02,2.020000e+03,2.000000e+00,6.000000e+00,1.600000e+01,4.000000e+00,1.980500e+04,2.294000e+03,1.997700e+04,2.294000e+03,1.288900e+04,1.288903e+06,3.145300e+04,2.600000e+01,4.500000e+01,1.288900e+04,1.288903e+06,3.145300e+04,2.600000e+01,4.500000e+01,0.000000e+00,-1.000000e+00,1.400000e+01,1.336000e+03,1.501000e+03,6.000000e+00,1.515000e+03,-7.000000e+00,0.000000e+00,-1.000000e+00,3.000000e+00,0.000000e+00
75%,2021-07-17 00:00:00,1.730000e+03,1.736000e+03,5.000000e+00,5.000000e+00,1.910000e+03,6.000000e+00,1.380000e+02,1.690000e+02,1.640000e+02,1.014000e+03,2.021000e+03,3.000000e+00,9.000000e+00,2.300000e+01,6.000000e+00,1.997700e+04,4.256000e+03,2.037800e+04,4.255000e+03,1.405700e+04,1.405702e+06,3.257500e+04,4.200000e+01,8.200000e+01,1.405700e+04,1.405702e+06,3.257500e+04,4.200000e+01,8.200000e+01,0.000000e+00,0.000000e+00,1.900000e+01,1.750000e+03,1.905000e+03,9.000000e+00,1.915000e+03,6.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00
max,2022-07-31 00:00:00,2.359000e+03,2.400000e+03,7.223000e+03,

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29193782 entries, 0 to 29193781
Data columns (total 61 columns):
 #   Column                                   Dtype         
---  ------                                   -----         
 0   FlightDate                               datetime64[us]
 1   Airline                                  object        
 2   Origin                                   object        
 3   Dest                                     object        
 4   Cancelled                                bool          
 5   Diverted                                 bool          
 6   CRSDepTime                               int64         
 7   DepTime                                  float64       
 8   DepDelayMinutes                          float64       
 9   DepDelay                                 float64       
 10  ArrTime                                  float64       
 11  ArrDelayMinutes                          float64       
 12  AirTime                   

***Coclusion:***

Nuestro data frame contiene los siguientes tipos de datos:

- ``Booleanos`` --> 2
- ``Date time`` --> 1
- ``Float``     --> 19
- ``Integer``   --> 22
- ``String``    --> 17



In [9]:
df.isna().sum()[df.isna().sum()>0]

DepTime                 761652
DepDelayMinutes         763084
DepDelay                763084
ArrTime                 786177
ArrDelayMinutes         846183
AirTime                 852561
CRSElapsedTime              22
ActualElapsedTime       845637
Tail_Number             267613
DepDel15                763084
DepartureDelayGroups    763084
TaxiOut                 780561
WheelsOff               780551
WheelsOn                793133
TaxiIn                  793143
ArrDelay                846183
ArrDel15                846183
ArrivalDelayGroups      846183
DivAirportLandings          90
dtype: int64

***Conclusion:***

De aproximadamente 29M de filas al rededor de 850k de ellas tienen valores nulos. Lo que significa que tenemos aproximadamente un 2,9% de valores nulos en toda nuestra base de datos

Sabiendo estos numeros lo que haremos seremos eliminarlos para que no afecte en el analisis de datos. Esta decision fue tomada considerando que al ser un porcentaje tan bajo no afectara al momento del analisis

## 2.3 Eliminating Irrelevant Features and dealing with null values

In [10]:
print("Hola Mundo")

Hola Mundo
